# Initial Data Exploration
This is going to be a little difficult for a couple of reasons

1. The data is poorly described, there is not accompanying readme or documentation on what each of the fields means. We are going to have to rely on the [original paper](https://www.researchgate.net/profile/Masako-Tamaki/publication/236113471_Neural_Decoding_of_Visual_Imagery_During_Sleep/links/02e7e53a5e1eba1005000000/Neural-Decoding-of-Visual-Imagery-During-Sleep.pdf) and its [supplemental materials](https://www.science.org/doi/suppl/10.1126/science.1234330/suppl_file/horikawa.sm.pdf) to figure out how to use it
2. The data is in an h5 format, which is something we haven't worked with before
3. The data is spread across several files, so we will have to aggregate it ourselves

In [9]:
!ls preproc

PreprocessedPerceptionDataSubject1.h5  PreprocessedSleepDataSubject3.h5
PreprocessedPerceptionDataSubject2.h5  propsSubject1.h5
PreprocessedPerceptionDataSubject3.h5  propsSubject2.h5
PreprocessedSleepDataSubject1.h5       propsSubject3.h5
PreprocessedSleepDataSubject2.h5


The files appear to be in "h5" format, which we can read using the h5py library ([documentation](https://docs.h5py.org/en/stable/))

It seems like there's 3 different classes of files, Perception data, Sleep data, and "props"?

The paper attempts to reconstruct visual data rather than just classify, so it uses a decoder trained from perception data while viewing particular images, most likely the "Perception" data, so we can probably ignore this. 

The sleep data probably has what we are looking for.

I'm not really sure what props is.

In [1]:
import h5py
import os

In [2]:
root = "preproc"
perc = os.path.join(root, "PreprocessedPerceptionDataSubject1.h5")
sleep = os.path.join(root, "PreprocessedSleepDataSubject1.h5")
props = os.path.join(root, "propsSubject1.h5")

Start with sleep data

In [3]:
dfile = h5py.File(sleep, "r")
dfile.keys()

<KeysViewHDF5 ['data', 'metaData', 'metaDefinition']>

Looks like theres 3 keys in the dataset, data, metadata, and metadefinition

In [4]:
for key in dfile.keys():
    dset = dfile[key]
    try:
        print(key, dset.shape, dset.dtype, dset[0], end="\n\n")
    except:
        print(key, dset, type(key), end="\n\n")

data (235, 4039) float64 [-1.23439404 -0.48702744  1.19918902 ...  1.          1.
  1.        ]

metaData <HDF5 group "/metaData" (43 members)> <class 'str'>

metaDefinition (43,) |S33 b'0 = not voxelData, 1 = voxelData'



In [6]:
tfile = h5py.File(os.path.join(root, "PreprocessedSleepDataSubject2.h5"), "r")
tfile["data"].shape

(198, 3981)

Looks like data is just a 2D array of data with where the first dimension is the number of "awakenings", which is confirmed by the paper...
> (235, 198, and
186 awakenings for participants 1 to 3, respec-
tively, used for decoding analyses) 

metaData is a group and metaDefinition contains some kind of string data, the "meta" datasets seem to be related since they have the same number of elements

In [7]:
data = dfile["data"]
mdata = dfile["metaData"]
mdef = dfile["metaDefinition"]
for i, key in enumerate(mdata):
    print(i, key, mdata[key].shape, " -- ",mdef[i])

0 EEG_sleep_score (1, 4039)  --  b'0 = not voxelData, 1 = voxelData'
1 FFA (1, 4039)  --  b'Value = X coordinate'
2 HVC (1, 4039)  --  b'Value = Y coordinate'
3 LOC (1, 4039)  --  b'Value = Z coordinate'
4 LVC (1, 4039)  --  b'0 = not FFA voxel, 1 = FFA voxel'
5 PPA (1, 4039)  --  b'0 = not HVC voxel, 1 = HVC voxel'
6 Synset_building_ID_02913152 (1, 4039)  --  b'0 = not LOC voxel, 1 = LOC voxel'
7 Synset_chair_ID_03001627 (1, 4039)  --  b'0 = not LVC voxel, 1 = LVC voxel'
8 Synset_character_ID_06818970 (1, 4039)  --  b'0 = not PPA voxel, 1 = PPA voxel'
9 Synset_clothing_ID_03051540 (1, 4039)  --  b'0 = not V1 voxel, 1 = V1 voxel'
10 Synset_code_ID_06355894 (1, 4039)  --  b'0 = not V2 voxel, 1 = V2 voxel'
11 Synset_cognition_ID_00023271 (1, 4039)  --  b'0 = not V3 voxel, 1 = V3 voxel'
12 Synset_external_body_part_ID_05225090 (1, 4039)  --  b'0 = not label, 1 = label'
13 Synset_geographical_area_ID_08574314 (1, 4039)  --  b'0 = absent, 1 = present'
14 Synset_girl_ID_10129825 (1, 4039)  -

Seem to be feature-value pairs, mapped to the data using masks?

Also, these seem to be out of order, most likely since dictionary indexing is unordered, but that shouldn't really be a problem since we have masks describing their regions

From the paper we know...
- LVC, HVC, V1, V2, V3, LOC, FF, and PPA are all visual areas. These will be our input
- Synsets are the different "classes" of things they saw in their dreams, these will be what we are trying to predict.

It looks like this is all we need, but I'm still going to take a look at "props" just to see what it is.

In [8]:
pfile = h5py.File(props, "r")
pfile.keys()

<KeysViewHDF5 ['roiMask', 'roiNames', 'synsetNames', 'synsetPairs', 'xyz']>

In [9]:
for key in pfile.keys():
    dset = pfile[key]
    try:
        print(key, dset.shape, dset.dtype, dset[0], end="\n\n")
    except:
        print(key, dset, type(key), end="\n\n")

roiMask (8, 4010) float64 [0. 0. 0. ... 0. 0. 0.]

roiNames (8,) |S4 b'FFA'

synsetNames (26,) |S38 b'Synset_male_ID_09624168'

synsetPairs (201, 2) |S38 [b'Synset_character_ID_06818970' b'Synset_male_ID_09624168']

xyz (3, 4010) float64 [-67.5 -67.5 -67.5 ...  64.5  64.5  64.5]



This seems to be the information on the boundaries between each of the features described in the "sleep" data. From the supplemental materials, the roi is "region of interest" which is the region of the brain.

In [10]:
rmask = pfile["roiMask"]
rnames = pfile["roiNames"]
xyz = pfile["xyz"]

print(rmask[0, :].sum(), rnames[0])
print(xyz[:, 2])

for region in range(len(rnames)):
    numel = rmask[region].sum() # number of "1" elements in the roi masks
    numel_d = sum(mdata[rnames[region]][0]) # number of "1" elements in corresponding mdata masks
    print(rnames[region], " -- ", numel, " -- ", numel_d) # should both be the same

537.0 b'FFA'
[-67.5 -43.5  -4.5]
b'FFA'  --  537.0  --  537.0
b'HVC'  --  1956.0  --  1956.0
b'LOC'  --  523.0  --  523.0
b'LVC'  --  2054.0  --  2054.0
b'PPA'  --  353.0  --  353.0
b'V1'  --  885.0  --  885.0
b'V2'  --  901.0  --  901.0
b'V3'  --  728.0  --  728.0


From the supplemental materials...
> For the analysis of individual subareas, the following numbers of voxels were identified
for V1, V2, V3, LOC, FFA, and PPA, respectively: 885, 901, 728, 523, 537, and 353 voxels for Subject 1;

This matches with the sum of our region of interest masks, meaning that these masks do in fact show the region of interest. The "xyz" feature appears to be the voxel (3d pixel) coordinates corresponding to each entry in our region of interest (which we don't really need)

In [11]:
print(data.shape)
ffa_mask = rmask[0].astype(bool)
print(ffa_mask.shape)

(235, 4039)
(4010,)


The mask seems to be shorter by 29 elements, which I would assume is the space for the meta elements?

In [12]:
data[-1, 4009:]

array([  1.97166996,   1.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   1.        ,
        26.        , 235.        ])

Yep! The last two seem to be numbers rather than logical values which matches what we would expect if `mdef` is properly ordered. Now we just need to extract all of the info into a more user-friendly form, probably pandas.

# Structure Data

In [13]:
import numpy as np
import pandas as pd

Pandas `read_hf` function apparently has problems, and I can't get it to read properly so I'm just going to create the dataframes manually

We can't really mask the data right now to extract features since we would end up with a ragged dataframe (different lengths for each column) which pandas doesn't like. Instead, I'll just make a dataframe of the masks and make the data into a numpy array.

In [14]:
dfile = h5py.File(sleep, "r") #data file

data = dfile["data"]
mdata = dfile["metaData"]

#convert from h5 to a dictionary of np arrays, which pandas understands
mdata_dict_arr = {key: np.squeeze(np.array(val)) for key, val in mdata.items()}

meta_df = pd.DataFrame.from_dict(mdata_dict_arr)
data_df = np.array(data)

In [18]:
print(meta_df.shape)
meta_df.head()

(4039, 43)


,EEG_sleep_score,FFA,HVC,LOC,LVC,PPA,Synset_building_ID_02913152,Synset_chair_ID_03001627,Synset_character_ID_06818970,Synset_clothing_ID_03051540,...,V2,V3,X,Y,Z,design,label,sample_number,session_number,voxelData
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-46.5,-1.5,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-46.5,1.5,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-43.5,-4.5,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-43.5,-1.5,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-64.5,-52.5,-4.5,0.0,0.0,0.0,0.0,1.0


In [19]:
print(data_df.shape)

(235, 4039)


In [21]:
data_df

array([[-1.23439404e+00, -4.87027437e-01,  1.19918902e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.63373625e+00,  5.24539088e+00, -1.69498203e+00, ...,
         1.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       [ 4.01640746e-01,  3.25192376e+00, -1.38049469e+00, ...,
         1.00000000e+00,  1.00000000e+00,  3.00000000e+00],
       ...,
       [-2.85550630e+00, -2.69055550e+00, -1.17711014e+00, ...,
         1.00000000e+00,  2.60000000e+01,  2.33000000e+02],
       [-1.89436357e-01, -1.59084747e+00,  2.13385080e+00, ...,
         1.00000000e+00,  2.60000000e+01,  2.34000000e+02],
       [-1.89219074e+00, -2.45376516e+00,  3.57550935e+00, ...,
         1.00000000e+00,  2.60000000e+01,  2.35000000e+02]])

Now that we're done with exploration, I'll add a module `utils` with the function `load_data` into the main project folder so that we can get these structures more easily. I added full docstrings, so it should be pretty easy to use, just take a look in the file for info on inputs and outputs.

In [24]:
import utils

masks, data =  utils.load_data(1)

In [25]:
masks.head()

,EEG_sleep_score,FFA,HVC,LOC,LVC,PPA,Synset_building_ID_02913152,Synset_chair_ID_03001627,Synset_character_ID_06818970,Synset_clothing_ID_03051540,...,V2,V3,X,Y,Z,design,label,sample_number,session_number,voxelData
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-46.5,-1.5,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-46.5,1.5,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-43.5,-4.5,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-67.5,-43.5,-1.5,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-64.5,-52.5,-4.5,0.0,0.0,0.0,0.0,1.0


In [26]:
print(f"data shape: {data.shape}")

data shape: (235, 4039)
